In [ ]:
## CAUTION: this packet is mandatory for Twint working ##

import nest_asyncio
nest_asyncio.apply()

# rest of the code
import twint
import pandas as pd
import preprocessor as p
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import seaborn as sns

import os 

import numpy as np

import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('always')

## Tweet Scrapping 

In [ ]:
def tweet_scrap(movie_name):
    
    c = twint.Config()
    #c.Username = "apple"
    c.Search =  "#{}".format(movie_name)
    #c.Search =  "#Mandalorien"
    #c.Format = "Tweet: {tweet}"
    c.Hide_output = True
    c.Lang = "en"
    c.Limit = 400
    c.Store_csv = True 
    ## Custom the output ##
    c.Custom["Tweet"] = ["tweet"]
    #c.Custom["Date"] = ["date"]

    c.Output = "{}.csv".format(movie_name)
    #c.Pandas = True
    
    twint.run.Search(c)
    warnings.filterwarnings('ignore')

## Tweet cleaning - Preprocess 

In [ ]:
def Clean(dataframe):
    #Remove non ASCII characters:
    dataframe = dataframe.encode('ascii','ignore')
    dataframe = dataframe.decode('utf8')
    
    #Remove URLs : 
    dataframe = re.sub('http\S+\s*','',dataframe)  
    
    
    #removing mentions:
    dataframe = re.sub('rt|cc','',dataframe) #Remove RT and cc
    dataframe = re.sub('#\S+','',dataframe) #Remove hashtags
    
    dataframe = dataframe.lower() #Converting to lowercase
    #dataframe = re.sub(r'\d+', '', dataframe) #Removing numbers
    dataframe = re.sub(r'[^a-zA-Z0-9\s]', '', dataframe) #Removing punctuations
    dataframe = dataframe.strip() #remove white spaces
    dataframe = re.sub('pictw\S+\s*','',dataframe)
    
    #Tokenize: 
    word_tokens = word_tokenize(dataframe)
    
    #Words stops
    stop_words=set(stopwords.words('english'))

    filtered_tweet = []
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
     
    return ' '.join(filtered_tweet)

In [ ]:
#df_tweet_clean = df_tweet['tweet'].apply(Clean)
#preproctest(df_tweet)

## Sentiment  Prediction with TextBlob

In [ ]:
from textblob import TextBlob
from IPython.display import Markdown, display
# Pretty printing the result
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
    
def Senti_Blob(df_tweet_clean):
    
    senti_blob = []
    for tweet in df_tweet_clean:
        #print(tweet)
        analysis = TextBlob(tweet)
        #print(analysis.sentiment)
        #senti.append(analysis.sentiment)
        if analysis.sentiment[0]>0:
            #printmd('Positive', color="green")
            senti_blob.append('positive')
        elif analysis.sentiment[0]<0:
            #printmd('Negative', color="red")
            senti_blob.append('negative')
        else:
            #printmd("Neutre", color="grey")
            senti_blob.append('neutre')
            #print("")

    File_sentiblob = {'Tweet': df_tweet_clean,
            'Sentiment': senti_blob,
           }

    df_sentiblob = pd.DataFrame(File_sentiblob, columns= ['Tweet', 'Sentiment'])
    
    return df_sentiblob

In [ ]:
#df_tweet['tweet'].count()

In [ ]:
#File_sentiblob = {'Tweet': df_tweet_clean,
#            'Sentiment': senti_blob,
#           }

#df_sentiblob = pd.DataFrame(File, columns= ['Tweet', 'Sentiment'])
#df_sentiblob   

## Predicted Emotion Analyses  

In [ ]:
#df_sentiblob.count()

In [ ]:
#sns.countplot(x='Sentiment',data=df_sentiblob)

In [ ]:
#count_sentiblob = df_sentiblob.groupby("Sentiment", sort='count').size().reset_index(name='count')
#print('\n Sentiment with Sentiblob: \n\n', count_sentiblob)

# Classifier

### Naive Bayes :

In [ ]:
def NB(X_train, X_test, Y_train):

    # fit the training dataset on the NB classifier
    Naive = MultinomialNB()
    Naive.fit(X_train, Y_train)

    # predict the labels on validation dataset
    predictions_NB = Naive.predict(X_test)
    
    #print(predictions_NB)
    print(f"\n NB CLASSIFIER: \n emotion positive : {list(predictions_NB).count('positive')} , negative : {list(predictions_NB).count('negative')} , neutral : {list(predictions_NB).count('neutral')} " )
    return predictions_NB

### SVM :

In [ ]:
def SVM(X_train, X_test, Y_train):
    
    SVM = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train, Y_train)

    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(X_test)

    #print(predictions_SVM)
    print(f"\n SVM CLASSIFIER: \n emotion positive : {list(predictions_SVM).count('positive')} , negative : {list(predictions_SVM).count('negative')} , neutral : {list(predictions_SVM).count('neutral')}" )
    return predictions_SVM 

### Random Forest Classifier

In [ ]:
def RFC(X_train, X_test, Y_train):
    
    RFC = RandomForestClassifier(n_estimators=1000, random_state=0)
    RFC.fit(X_train, Y_train)

    # predict the labels on validation dataset
    predictions_RFC = RFC.predict(X_test)

    print(f"\n RFC CLASSIFIER: \n emotion positive : {list(predictions_RFC).count('positive')} , negative : {list(predictions_RFC).count('negative')}, neutral : {list(predictions_RFC).count('neutral')} " )
    return predictions_RFC 

### Logistic Regression

In [ ]:
def LR(X_train, X_test, Y_train ):
    
    logreg = LogisticRegression(solver='liblinear', multi_class='auto')
    logreg.fit(X_train, Y_train)

    # predict the labels on validation dataset
    predictions_LR = logreg.predict(X_test)

    print(f"\n Logistic Regression CLASSIFIER: \n emotion positive : {list(predictions_LR).count('positive')} , negative : {list(predictions_LR).count('negative')} , neutral : {list(predictions_LR).count('neutral')}" )
    return predictions_LR 

# API

In [ ]:
def Api(movie_name):
    tweet_scrap(movie_name)  ### A commenter pour juste prendre les csv pre enregistres
    df_tweet = pd.read_csv("{}.csv".format(movie_name))
    df_tweet_clean = df_tweet['tweet'].apply(Clean)
    #Senti_Blob(df_tweet_clean)
    df_sentiment_blob = Senti_Blob(df_tweet_clean)
    print('\n \n')
    print(df_sentiment_blob)
    
    count_senti = df_sentiment_blob.count()
    print('\n \n')
    print('Total number of tweet: \n',count_senti)
    
    sns.countplot(x='Sentiment', data=df_sentiment_blob)
    
    count_sentiblob = df_sentiment_blob.groupby("Sentiment", sort='count').size().reset_index(name='count')
    print('\n #####  Sentiment with Sentiblob: ###### \n\n', count_sentiblob)
    
    ## Pourcentage 
    
    nb_tot = df_sentiment_blob['Sentiment'].shape[0]
    positive_sentiblob = list(df_sentiment_blob['Sentiment']).count('positive')
    negative_sentiblob = list(df_sentiment_blob['Sentiment']).count('negative')
    neutral_sentiblob = list(df_sentiment_blob['Sentiment']).count('neutre')

    print('\n percent of positive:', (positive_sentiblob/nb_tot)*100)
    print( '\n percent of negative:', (negative_sentiblob/nb_tot)*100)
    print('\n percent of neutre:', (neutral_sentiblob/nb_tot)*100)
    
    
    #### PLOT pourcent Senti BLOB ##
    
    labels = ['positive', 'neutral', 'negative']
    pred_NB = [(positive_sentiblob/nb_tot)*100, (neutral_sentiblob/nb_tot)*100, (negative_sentiblob/nb_tot)*100 ]
    

    x = np.arange(len(labels))  # the label locationsc
    width = 0.45  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x , pred_NB, width)
   
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Percent')
    ax.set_title('Sentiment Percentage Prediction with SentiBlob')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)

    fig.tight_layout()

    plt.show()
    
    #######################################
    
    
    data_clean = df_tweet_clean
    
    ## Training file
    train_data = pd.read_csv('train.csv', encoding='utf-8')
    train_data = train_data[['Category', 'Tweet']]
    train_data.rename(columns={'Category': 'emotion'}, inplace=True)
    train_data = train_data[train_data['emotion'] != 'Tweet']
    train_data = train_data[train_data['Tweet'] != "Not Available"]
    train_data_clean = train_data['Tweet'].apply(Clean)
    
    
    sns.countplot(x='emotion',data=train_data)
    
    #Vectorizer : 
    count_vectorizer = CountVectorizer()
    X_train = count_vectorizer.fit_transform(train_data_clean)
    Y_train = train_data['emotion']
    X_test = count_vectorizer.transform(data_clean)

    #print(X_train.shape, Y_train.shape, X_test.shape)
    
    print('\n #####  Sentiment Prediction with CLASSIFIER : ###### \n\n')
    
    ## Classifier ####
    
    predictions_NB = NB(X_train, X_test, Y_train)
    predictions_SVM = SVM(X_train, X_test, Y_train)
    predictions_RFC = RFC(X_train, X_test, Y_train)
    predictions_LR = LR(X_train, X_test, Y_train)
    
    ### Percent calcul
    nb_NB = predictions_NB.shape[0]
    positive_NB = list(predictions_NB).count('positive')
    negative_NB = list(predictions_NB).count('negative')
    neutral_NB = list(predictions_NB).count('neutral')
    
    positive_NBp = (positive_NB/nb_NB)*100
    negative_NBp = (negative_NB/nb_NB)*100
    neutral_NBp = (neutral_NB/nb_NB)*100
    
    print("\n NB :")
    print( 'Positive:', positive_NBp, '\n Negative:', negative_NBp, '\n Neutre:', neutral_NBp)

    nb_SVM = predictions_SVM.shape[0]
    positive_SVM = list(predictions_SVM).count('positive')
    negative_SVM = list(predictions_SVM).count('negative')
    neutral_SVM = list(predictions_SVM).count('neutral')
    
    positive_SVMp = (positive_SVM/nb_SVM)*100
    negative_SVMp = (negative_SVM/nb_SVM)*100
    neutral_SVMp = (neutral_SVM/nb_SVM)*100
    
    print("\n SVM :")
    print( 'Positive:', positive_SVMp , '\n Negative:', negative_SVMp , '\n Neutre:', neutral_SVMp)
    
    nb_RFC = predictions_RFC.shape[0]
    positive_RFC = list(predictions_RFC).count('positive')
    negative_RFC = list(predictions_RFC).count('negative')
    neutral_RFC = list(predictions_RFC).count('neutral')
    
    
    positive_RFCp = (positive_RFC/nb_RFC)*100
    negative_RFCp = (negative_RFC/nb_RFC)*100
    neutral_RFCp = (neutral_RFC/nb_RFC)*100
    
    print("\n RFC :")
    print( 'Positive:', positive_RFCp, '\n Negative:', negative_RFCp, '\n Neutre:', neutral_RFCp)

    nb_LR = predictions_LR.shape[0]
    positive_LR = list(predictions_LR).count('positive')
    negative_LR = list(predictions_LR).count('negative')
    neutral_LR = list(predictions_LR).count('neutral')
    
    positive_LRp = (positive_LR/nb_LR)*100
    negative_LRp = (negative_LR/nb_LR)*100
    neutral_LRp = (neutral_LR/nb_LR)*100
    
    
    print("\n LR :")
    print( 'Positive:', positive_LRp, '\n Negative:', negative_LRp, '\n Neutre:', neutral_LRp)
    

    aver_pos = (((positive_NB/nb_NB)+(positive_SVM/nb_SVM)+(positive_LR/nb_LR))/3)*100
    aver_neg = (((negative_NB/nb_NB)+(negative_SVM/nb_SVM)+(negative_LR/nb_LR))/3)*100
    aver_neut = (((neutral_NB/nb_NB)+(neutral_SVM/nb_SVM)+(neutral_LR/nb_LR))/3)*100
   
    print("\n\n Average :")
    print( '\n Positive:', aver_pos, '\n Negative:',aver_neg, '\n Neutre:', aver_neut)
    
    
    
    #### PLOT  CLASSIFIER COMPARAISON  ####
    
    labels = ['positive', 'negative', 'neutral']
    pred_NB = [positive_NBp, negative_NBp, neutral_NBp]
    pred_SVM = [positive_SVMp, negative_SVMp, neutral_SVMp]
    pred_RFC = [positive_RFCp, negative_RFCp, neutral_RFCp]
    pred_LR = [positive_LRp, negative_LRp, neutral_LRp]

    x = np.arange(len(labels))  # the label locationsc
    width = 0.45  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, pred_NB, width/4, label='NB')
    rects2 = ax.bar(x - width/4, pred_SVM, width/4, label='SVM')
    rects3 = ax.bar(x + width/2, pred_RFC, width/4, label='RFC')
    rects4 = ax.bar(x + width/4, pred_LR, width/4, label='LR')
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Percent')
    ax.set_title('Sentiment Percentage Prediction for each classifier')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    fig.tight_layout()

    plt.show()
    
    
     #### PLOT  AVERAGE  ####
       
    labels = ['positive', 'negative', 'neutral']
    pred_NB = [aver_pos, aver_neg, aver_neut ]
    

    x = np.arange(len(labels))  # the label locationsc
    width = 0.45  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x , pred_NB, width)
   
    
    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Percent')
    ax.set_title(' Average: Sentiment Percentage Prediction ')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)

    fig.tight_layout()

    plt.show()
    
    return aver_pos
    

In [ ]:
movie_name = "Spectre"
#movie_name = "avengersendgame"
#movie_name = "peakyblinders"
#Api(movie_name)


#movie_name = "hobgoblins"
aver_mov = Api(movie_name)

In [ ]:
print(aver_mov)

In [ ]:
import imdb 
import pandas as pd 
import re  

ia = imdb.IMDb()
movie_list=[]

for name in ('8111088', '0068646', '0089280', '5690360', '10326928', '6105098', '8380776', '2379713'):
    movie = ia.get_movie(name) 
    name = movie['title']
    score = movie.get('rating')
    #print("Name :",name,", Score :", score)
    movie_list.append((name,score))

df = pd.DataFrame(movie_list, columns = ['Name', 'Score']) 

def Clean_title(data):
    data=data.lower()
    data = data.replace(' ', '')
    data= re.sub(r'[^a-zA-Z0-9\s]', '', data)
    return data

df_name_clean = df['Name'].apply(Clean_title)


j=0
for i in df_name_clean:
    print(" --------------------- the analyses for the movie/serie:", df['Name'][j], "is : ")
    aver_mov = Api(i)
    print("The score for the movie/serie ",df['Name'][j] ,"from iMDb is: ",df['Score'][j], "\n\n ")
    j=j+1
    
    